## Import model

In [2]:
## import libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets

from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.tensorboard import SummaryWriter

CUDA = torch.cuda.is_available()
# device = "cpu"
device = torch.device("cuda" if CUDA else "cpu")

## Prep dataset

In [3]:
import sys
# run the below line once only
# sys.path.insert(0,r'C:\Users\siyang\Documents\GitHub\chexnet')
print(sys.path)

['C:\\Users\\siyang\\Documents\\GitHub\\chexnet', 'c:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject\\notebooks', 'C:\\Python312\\python312.zip', 'C:\\Python312\\DLLs', 'C:\\Python312\\Lib', 'C:\\Python312', 'c:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject\\.venv', '', 'c:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject\\.venv\\Lib\\site-packages', 'C:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject', 'c:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject\\.venv\\Lib\\site-packages\\win32', 'c:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject\\.venv\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject\\.venv\\Lib\\site-packages\\Pythonwin']


In [4]:
from DatasetGenerator import DatasetGenerator

In [5]:
pathDirData = r'C:\Users\siyang\Documents\GitHub\DeepLearningProject\raw_data\archive'
pathFileTrain = r'C:\Users\siyang\Documents\GitHub\DeepLearningProject\chexnet\dataset\train_1.txt'
pathFileVal = r'C:\Users\siyang\Documents\GitHub\DeepLearningProject\chexnet\dataset\val_1.txt'

transResize = 256
transCrop = 224
trBatchSize = 128
num_class = 14

normalize = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

transformList = []
# transformList.append(transforms.Resize(transResize))
transformList.append(transforms.RandomResizedCrop(transCrop))
transformList.append(transforms.RandomHorizontalFlip())
transformList.append(transforms.ToTensor())
transformList.append(normalize)      
transformSequence=transforms.Compose(transformList)

datasetTrain = DatasetGenerator(pathImageDirectory=pathDirData, pathDatasetFile=pathFileTrain, transform=transformSequence)
datasetVal =   DatasetGenerator(pathImageDirectory=pathDirData, pathDatasetFile=pathFileVal, transform=transformSequence)
train_loader = DataLoader(dataset=datasetTrain, batch_size=trBatchSize, shuffle=True,  num_workers=12, pin_memory=True)
val_loader = DataLoader(dataset=datasetVal, batch_size=trBatchSize, shuffle=False, num_workers=12, pin_memory=True)

## Call MaxVit Model

In [6]:
from torchvision.models import maxvit

In [7]:
## Initialise first layer
model = maxvit.maxvit_t(
    weights = maxvit.MaxVit_T_Weights,
    input_size = (transCrop, transCrop)
)
##  Freeze earlier weights
for param in model.parameters():
    param.requires_grad= False

##  Replace the last layer 
model.classifier[-1] = nn.Sequential(
    nn.Linear(in_features=model.classifier[5].in_features,out_features= num_class),
    nn.Softmax(dim=1)
)

## set to cuda
model.cuda()

c:\Users\siyang\Documents\GitHub\DeepLearningProject\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MaxVit_T_Weights.IMAGENET1K_V1`. You can also use `weights=MaxVit_T_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\siyang\Documents\GitHub\DeepLearningProject\.venv\Lib\site-packages\torch\functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3550.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


MaxVit(
  (stem): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
      (2): GELU(approximate='none')
    )
    (1): Conv2dNormActivation(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
  (blocks): ModuleList(
    (0): MaxVitBlock(
      (layers): ModuleList(
        (0): MaxVitLayer(
          (layers): Sequential(
            (MBconv): MBConv(
              (proj): Sequential(
                (0): AvgPool2d(kernel_size=3, stride=2, padding=1)
                (1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
              )
              (stochastic_depth): Identity()
              (layers): Sequential(
                (pre_norm): BatchNorm2d(64, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
                (conv_a): Conv2dNormActivation(
           

Model Training

In [8]:
model.classifier[-3].weight.requires_grad

False

In [9]:
## function to calculate the F1 score
def f1_score(tp, fp, fn):
    return 2 * (tp) / (2 * tp + fp + fn)

In [10]:
# Define the loss function and optimizer
criterion = nn.BCELoss(reduction='mean')
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)
scheduler = ReduceLROnPlateau(optimizer, factor = 0.1, patience = 5, mode = 'min')

# Create a TensorBoard writer
writer = SummaryWriter()

# Train the model
n_epochs = 5
for epoch in range(n_epochs):

    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # calculate statistics
        # pred_labels = (nn.Softmax(dim=1)(outputs) > 1/14).long()
        
        tp_array = [0 for x in range(num_class)]
        fp_array = [0 for x in range(num_class)]
        fn_array = [0 for x in range(num_class)]
        pred_labels = (outputs > 1/4).long()
        tp_array += sum(torch.logical_and(pred_labels, labels))
        fp_array += sum(torch.logical_and(torch.logical_xor(pred_labels, labels).long(), pred_labels))
        fn_array += sum(torch.logical_and(torch.logical_xor(pred_labels, labels).long(), labels))
        
        writer.add_scalar('Loss/train', loss, epoch * len(train_loader) + i)
        writer.add_scalar('TP_Sum/train', sum(tp_array), epoch * len(train_loader) + i)
        writer.add_scalar('FP_Sum/train', sum(fp_array), epoch * len(train_loader) + i)
        writer.add_scalar('FN_Sum/train', sum(fn_array), epoch * len(train_loader) + i)
        writer.add_scalar('F1_Score/train', f1_score(sum(tp_array), sum(fp_array), sum(fn_array)), epoch * len(train_loader) + i)

        # Backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Display
        # if (i + 1) % 100 == 0:
        print("Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, tp_sum: {:.4f}, fp_sum: {:.4f}, fn_sum: {:.4f}, cumulative_f1_score: {:.4f}".format(epoch + 1, \
                                                                     n_epochs, \
                                                                     i + 1, \
                                                                     len(train_loader), \
                                                                     loss,\
                                                                     sum(tp_array), \
                                                                     sum(fp_array),\
                                                                     sum(fn_array),\
                                                                     f1_score(sum(tp_array), sum(fp_array), sum(fn_array))))

Epoch [1/5], Step [1/614], Loss: 0.2049, tp_sum: 0.0000, fp_sum: 0.0000, fn_sum: 90.0000, cumulative_f1_score: 0.0000
Epoch [1/5], Step [2/614], Loss: 0.2065, tp_sum: 0.0000, fp_sum: 1.0000, fn_sum: 98.0000, cumulative_f1_score: 0.0000
Epoch [1/5], Step [3/614], Loss: 0.1919, tp_sum: 11.0000, fp_sum: 74.0000, fn_sum: 81.0000, cumulative_f1_score: 0.1243
Epoch [1/5], Step [4/614], Loss: 0.1945, tp_sum: 20.0000, fp_sum: 113.0000, fn_sum: 73.0000, cumulative_f1_score: 0.1770
Epoch [1/5], Step [5/614], Loss: 0.1868, tp_sum: 18.0000, fp_sum: 81.0000, fn_sum: 72.0000, cumulative_f1_score: 0.1905
Epoch [1/5], Step [6/614], Loss: 0.2081, tp_sum: 19.0000, fp_sum: 73.0000, fn_sum: 87.0000, cumulative_f1_score: 0.1919
Epoch [1/5], Step [7/614], Loss: 0.1796, tp_sum: 23.0000, fp_sum: 70.0000, fn_sum: 64.0000, cumulative_f1_score: 0.2556
Epoch [1/5], Step [8/614], Loss: 0.1839, tp_sum: 20.0000, fp_sum: 79.0000, fn_sum: 76.0000, cumulative_f1_score: 0.2051
Epoch [1/5], Step [9/614], Loss: 0.2165, tp

KeyboardInterrupt: 